In [2]:
import psycopg2
conn = psycopg2.connect('postgresql://brosra01:@localhost/world')
cur = conn.cursor()
cur.execute('select * from country limit 10')
res = cur.fetchall()
#for row in res[:5]:
    #print(row)
    

In [3]:
import csv
import psycopg2
conn = psycopg2.connect('postgresql://brosra01:@localhost/world')
cur = conn.cursor()
x = 'USA'
cur.execute("select code, name, region, continent from country where code = '%s'" % x)
res = cur.fetchall()
#for row in res:
 #   print(row)

    
        

In [4]:
import csv
import psycopg2

with open('/home/faculty/millbr02/pub/un-general-debates.csv','r',encoding='latin-1') as data:
    csv_iter = csv.reader(data)
    db =[]
    for row in csv_iter:
        db.append(row)
        
        
db[:5]

[['session', 'year', 'country', 'text'],
 ['44',
  '1989',
  'MDV',
  "ï»¿It is indeed a pleasure for me and the members of my delegation to extend to Ambassador Garba our sincere congratulations on his election to the presidency of the forty-fourth session of the General Assembly. His election to this high office is a well-deserved tribute to his personal qualities and experience. I am fully confident that under his able and wise leadership the Assembly will further consolidate the gains achieved during the past year.\nMy delegation associates itself with previous speakers in expressing its appreciation of the dedicated efforts of his predecessor, His Excellency Mr. Dante Caputo, for the exemplary manner in which he discharged his duties as President of the forty-third session of the General Assembly.\nAs in previous years, my delegation wishes to note its satisfaction with and gratitude for the assiduous and unrelenting efforts exerted by the Secretary-General of the United Nations i

In [5]:
import csv
import psycopg2
with open('/home/faculty/millbr02/pub/un-general-debates.csv','r',encoding='latin-1') as data:
    csv_iter = csv.reader(data)
    db = []
    for row in csv_iter:
        db.append(row)
        
unknown_country = []
other_countries = {'CSK': ['Czechoslovakia', 'Europe', 'Eastern Europe'],
                   'DDR': ['East Germany', 'Europe', 'Western Europe'],
                   'EU': ['European Union', 'Europe', 'Western Europe'],
                   'MNE': ['Montenegro', 'Europe', 'Eastern Europe'],
                   'ROU': ['Romania', 'Europe', 'Eastern Europe'],
                   'SSD': ['South Sudan', 'Africa', 'Northern Africa'],
                   'TLS': ['East Timor', 'Asia', 'Southeast Asia'],
                   'YDYE': ['Democratic Yemen', 'Asia', 'Middle East']}

for row in db[1:]:
    cur.execute("select code, name, region, continent from country where code = %s", [row[2]])
    country = cur.fetchone()
    if country:
        row.append(country[1])
        row.append(country[2])
        row.append(country[3])
        
    else:
        print("No Entry for {}".format(row[2]))
        unknown_country.append(row[2])
        row.append(other_countries[row[2]][0])
        row.append(other_countries[row[2]][2])
        row.append(other_countries[row[2]][1])


No Entry for YDYE
No Entry for CSK
No Entry for ROU
No Entry for DDR
No Entry for EU
No Entry for TLS
No Entry for MNE
No Entry for ROU
No Entry for SSD
No Entry for ROU
No Entry for DDR
No Entry for CSK
No Entry for YDYE
No Entry for MNE
No Entry for ROU
No Entry for TLS
No Entry for ROU
No Entry for CSK
No Entry for CSK
No Entry for ROU
No Entry for YDYE
No Entry for DDR
No Entry for ROU
No Entry for CSK
No Entry for ROU
No Entry for DDR
No Entry for YDYE
No Entry for ROU
No Entry for ROU
No Entry for SSD
No Entry for MNE
No Entry for EU
No Entry for TLS
No Entry for ROU
No Entry for YDYE
No Entry for ROU
No Entry for CSK
No Entry for DDR
No Entry for DDR
No Entry for ROU
No Entry for CSK
No Entry for YDYE
No Entry for TLS
No Entry for MNE
No Entry for ROU
No Entry for DDR
No Entry for YDYE
No Entry for CSK
No Entry for ROU
No Entry for ROU
No Entry for ROU
No Entry for MNE
No Entry for TLS
No Entry for CSK
No Entry for YDYE
No Entry for ROU
No Entry for YDYE
No Entry for CSK
No Entr

In [6]:
#for yr in sorted(years):
 #   for region in sorted(years[yr]):
  #      print(yr, region, years[yr][region])

In [7]:
'''years = {}
for row in db[1:]:
    yr = row[1]
    country = row[2]
    if country not in unknown_country:
        region = row[5]
        if yr not in years:
            years[yr] = {}
        if region not in years[yr]:
            years[yr][region] = 1
        else:
            years[yr][region] += 1'''

'years = {}\nfor row in db[1:]:\n    yr = row[1]\n    country = row[2]\n    if country not in unknown_country:\n        region = row[5]\n        if yr not in years:\n            years[yr] = {}\n        if region not in years[yr]:\n            years[yr][region] = 1\n        else:\n            years[yr][region] += 1'

In [8]:
'''for yr in sorted(years):
    for region in sorted(years[yr]):
        print(yr, region, years[yr][region])

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-8-c0922e3dfbf1>, line 3)

In [10]:
with open('/home/faculty/millbr02/pub/un-debates-augmented-1.csv','r',encoding='latin-1') as data:
    csv_iter = csv.reader(data)
    db = []
    for row in csv_iter:
        db.append(row)
        

    country_names = {}
    for line in db[1:]:
        country_names[line[4]] = 0
        
        
    
    matrix = {}
    for row in db[1:]:
        spkr = row[4]
        text = row[3]
        if spkr not in matrix:
            matrix[spkr] = country_names.copy()
            
            
        
        for key in matrix:
            counter = text.count(key)
            matrix[spkr][key] += counter

        
                
                

matrix

            


                    
                    
        


  

    


{'Afghanistan': {'Afghanistan': 1398,
  'Albania': 1,
  'Algeria': 3,
  'Andorra': 0,
  'Angola': 19,
  'Antigua and Barbuda': 0,
  'Argentina': 2,
  'Armenia': 0,
  'Australia': 3,
  'Austria': 1,
  'Azerbaijan': 1,
  'Bahamas': 2,
  'Bahrain': 1,
  'Bangladesh': 6,
  'Barbados': 0,
  'Belarus': 0,
  'Belgium': 0,
  'Belize': 0,
  'Benin': 0,
  'Bhutan': 1,
  'Bolivia': 1,
  'Bosnia and Herzegovina': 5,
  'Botswana': 2,
  'Brazil': 0,
  'Brunei': 0,
  'Bulgaria': 2,
  'Burkina Faso': 0,
  'Burundi': 0,
  'Cambodia': 1,
  'Cameroon': 0,
  'Canada': 5,
  'Cape Verde': 1,
  'Central African Republic': 0,
  'Chad': 0,
  'Chile': 5,
  'China': 34,
  'Colombia': 1,
  'Comoros': 0,
  'Congo': 0,
  'Congo, The Democratic Republic of the': 0,
  'Costa Rica': 1,
  'Croatia': 0,
  'Cuba': 8,
  'Cyprus': 34,
  'Czech Republic': 0,
  'Czechoslovakia': 1,
  'CÃ´te dÂ\x92Ivoire': 0,
  'Democratic Yemen': 0,
  'Denmark': 1,
  'Djibouti': 2,
  'Dominica': 0,
  'Dominican Republic': 0,
  'East Germany'

In [ ]:
%matplotlib inline
import networkx as nx
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
g = nx.Graph()

In [ ]:
g.add_nodes_from(matrix) 
for key in matrix: 
    for key2 in matrix[key]: 
        if matrix[key][key2] > 50:
            g.add_edge(key, key2, weight=matrix[key][key2]) 


In [ ]:
gfig, gax = plt.subplots(figsize = (50,50))
hego = nx.ego_graph(g,'United States', radius =2)
pos = nx.spring_layout(g, k=.3, iterations=2000, scale =10)
nx.draw(hego, pos, ax=gax, node_size=200, with_labels=True, font_size=40)